In [24]:
import pandas as pd

In [25]:
df_stock_exchange = pd.read_excel('../data/raw/borsa_ist.xlsx')
df_bitcoin = pd.read_excel('../data/raw/btc.xlsx')
df_gold = pd.read_excel('../data/raw/gold_prices.xlsx')
df_gov_bonds = pd.read_excel('../data/raw/gov_bonds.xlsx')
df_hpi = pd.read_excel('../data/raw/hpi.xlsx')
df_cpi = pd.read_csv('../data/raw/turkish_inflation_cpi.csv', sep=";")

# Stock_Exchange:

In [26]:
df_stock_exchange = df_stock_exchange.iloc[1:].reset_index(drop=True)
df_stock_exchange.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df_stock_exchange.rename(columns={'Last Price (TRY)': 'Price_Stock_Ex'}, inplace=True)
df_stock_exchange['Date'] = pd.to_datetime(df_stock_exchange['Date'], errors='coerce')
df_stock_exchange['Date'] = df_stock_exchange['Date'].dt.strftime('%Y-%m')

df_stock_exchange['stock_return_mo'] = df_stock_exchange['Price_Stock_Ex'] / df_stock_exchange['Price_Stock_Ex'].shift(-1) - 1 #calculate 1 month stock return
df_stock_exchange['stock_return_pa'] = (1 + df_stock_exchange['stock_return_mo'])**(12) - 1 #calculate stock return per annum

df_stock_exchange
df_stock_exchange.to_csv('../data/processed/df_stock_exchange.csv', index=False)

# Bitcoin:

In [27]:
df_bitcoin = df_bitcoin.iloc[1:].reset_index(drop=True)
df_bitcoin.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df_bitcoin.rename(columns={'Unnamed: 1': 'Price_Bitcoin'}, inplace=True)
df_bitcoin['Date'] = pd.to_datetime(df_bitcoin['Date'], errors='coerce')
df_bitcoin['Date'] = df_bitcoin['Date'].dt.strftime('%Y-%m')

df_bitcoin['btc_return_mo'] = df_bitcoin['Price_Bitcoin'] / df_bitcoin['Price_Bitcoin'].shift(-1) -1 # calculate 1 month return
df_bitcoin['btc_return_pa'] = (1 + df_bitcoin['btc_return_mo'])**(12) - 1 #calculate return per annum

df_bitcoin
df_bitcoin.to_csv('../data/processed/df_bitcoin.csv', index=False)

# Gold:

In [28]:
df_gold = df_gold.iloc[1:].reset_index(drop=True)
df_gold.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df_gold.rename(columns={'Last Price (TRY)': 'Price_Gold'}, inplace=True)
df_gold['Date'] = pd.to_datetime(df_gold['Date'], errors='coerce')
df_gold['Date'] = df_gold['Date'].dt.strftime('%Y-%m')

df_gold['gold_return_mo'] = df_gold['Price_Gold'] / df_gold['Price_Gold'].shift(-1) -1 # calculate 1 month return
df_gold['gold_return_pa'] = (1 + df_gold['gold_return_mo'])**(12) - 1 #calculate return per annum (annualized return)

df_gold
df_gold.to_csv('../data/processed/df_gold.csv', index=False)

# Government Bonds:

In [29]:
df_gov_bonds = df_gov_bonds.iloc[1:].reset_index(drop=True)
df_gov_bonds.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
df_gov_bonds.rename(columns={'Yield (%)': 'Gov_Bond_Yield'}, inplace=True)
df_gov_bonds['Date'] = pd.to_datetime(df_gov_bonds['Date'], errors='coerce')
df_gov_bonds['Date'] = df_gov_bonds['Date'].dt.strftime('%Y-%m')  

df_gov_bonds['bond_return_pa'] = df_gov_bonds['Gov_Bond_Yield'] / 100

df_gov_bonds
df_gov_bonds.to_csv('../data/processed/df_gov_bonds.csv', index=False)

# House Price Index:

In [30]:
df_hpi = df_hpi.iloc[1:].reset_index(drop=True)

df_hpi.rename(columns={'Tarih': 'Date'}, inplace=True)
df_hpi['Date'] = pd.to_datetime(df_hpi['Date'], errors='coerce')
df_hpi['Date'] = df_hpi['Date'].dt.strftime('%Y-%m')

df_hpi = df_hpi.dropna(subset=['Date'])

df_hpi['hpi_return_mo'] = df_hpi['TP KFE TR-1'] / 100
df_hpi['hpi_return_pa'] = (1 + df_hpi['hpi_return_mo'])**(12) - 1 #calculate annualized return

#df_hpi
df_hpi.to_csv('../data/processed/df_hpi.csv', index=False)

# Consumer Price Index

In [31]:
df_cpi = df_cpi.iloc[1:].reset_index(drop=True)

df_cpi.rename(columns={'in %': 'cpi_inflation'}, inplace=True)
df_cpi['Date'] = pd.to_datetime(df_cpi['Date'], errors='coerce', dayfirst=True)
df_cpi['Date'] = df_cpi['Date'].dt.strftime('%Y-%m') #inflation measured at the end of the month

df_cpi['cpi_inflation'] = df_cpi['cpi_inflation'] / 100 

#df_cpi
df_cpi.to_csv('../data/processed/df_cpi.csv', index=False)

# Database:

In [32]:
import sqlite3

In [38]:
conn = sqlite3.connect('../data/processed/database_name.db')

In [40]:
df_stock_exchange.to_sql('STOCKEX', conn, if_exists='replace', index=False)
df_bitcoin.to_sql('BTC', conn, if_exists='replace', index=False)
df_gold.to_sql('GOLD', conn, if_exists='replace', index=False)
df_gov_bonds.to_sql('BOND', conn, if_exists='replace', index=False)
df_hpi.to_sql("HPI",conn, if_exists='replace', index=False)

152

In [42]:
query = "SELECT * FROM STOCKEX"
TRIAL = pd.read_sql_query(query, conn)
TRIAL

,Date,Price_Stock_Ex,stock_return_mo,stock_return_pa
0,2024-08,9833.22,-0.075702,-0.611183
1,2024-07,10638.58,-0.000876,-0.010464
2,2024-06,10647.91,0.023790,0.325964
3,2024-05,10400.48,0.035312,0.516552
4,2024-04,10045.74,0.098808,2.097851
...,...,...,...,...
75,2018-05,1006.52,-0.034819,-0.346408
76,2018-04,1042.83,-0.092639,-0.688569
77,2018-03,1149.30,-0.033804,-0.338113
78,2018-02,1189.51,-0.004836,-0.056509
